In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import random
from collections import deque


In [2]:
# Load dataset
df = pd.read_csv('surabaya_routes_with_vehicle_times.csv')

# Preprocess dataset
df['Traffic_Condition'] = df['Traffic_Condition'].map({'lancar': 0, 'sedang': 1, 'macet': 2})

# Normalize numerical data
df['Distance_km'] = df['Distance_km'] / df['Distance_km'].max()
df['Travel_Time_Mobil_min'] = df['Travel_Time_Mobil_min'] / df['Travel_Time_Mobil_min'].max()
df['Travel_Time_Motor_min'] = df['Travel_Time_Motor_min'] / df['Travel_Time_Motor_min'].max()


In [3]:
class RouteEnvironment:
    def __init__(self, data):
        self.data = data
        self.locations = data['Start_Name'].unique()
        self.reset()
    
    def reset(self):
        self.current_location = random.choice(self.locations)
        self.destination = random.choice([loc for loc in self.locations if loc != self.current_location])
        self.steps = 0
        return self.get_state()
    
    def get_state(self):
        state = self.data[(self.data['Start_Name'] == self.current_location) & 
                          (self.data['End_Name'] == self.destination)]
        if not state.empty:
            return np.array([state['Traffic_Condition'].values[0], 
                             state['Distance_km'].values[0]])
        else:
            return np.zeros(2)
    
    def step(self, action):
        # Action 0: Mobil, Action 1: Motor
        vehicle_col = 'Travel_Time_Mobil_min' if action == 0 else 'Travel_Time_Motor_min'
        state = self.data[(self.data['Start_Name'] == self.current_location) & 
                          (self.data['End_Name'] == self.destination)]
        if not state.empty:
            travel_time = state[vehicle_col].values[0]
            reward = -travel_time  # Negative reward for higher travel time
            done = True
            return self.get_state(), reward, done
        else:
            return self.get_state(), -1, True  # Penalize invalid moves


In [4]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()
    
    def _build_model(self):
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(24, input_dim=self.state_size, activation='relu'),
            tf.keras.layers.Dense(24, activation='relu'),
            tf.keras.layers.Dense(self.action_size, activation='linear')
        ])
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=self.learning_rate),
                      loss='mse')
        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])
    
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma *
                          np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon_decay = 0.99 


In [5]:
# Initialize environment and agent
env = RouteEnvironment(df)
state_size = 2  # Traffic_Condition and Distance_km
action_size = 2  # Mobil or Motor
agent = DQNAgent(state_size, action_size)
episodes = 200  # Reduce the number of episodes
batch_size = 32

for e in range(episodes):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    
    for time in range(200):  # Optional: Reduce max steps per episode
        action = agent.act(state)
        next_state, reward, done = env.step(action)
        next_state = np.reshape(next_state, [1, state_size])
        
        # Clipping reward for stability
        reward = max(min(reward, 1), -1)
        
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        
        if done:
            print(f"Episode: {e+1}/{episodes}, Reward: {reward}, Epsilon: {agent.epsilon:.2}")
            break
    
    # Replay only if we have enough memory
    if len(agent.memory) > batch_size:
        agent.replay(batch_size)

    # Optional: Save the model periodically
    if (e + 1) % 50 == 0:
        agent.model.save(f"dqn_model_episode_{e+1}.h5")


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Episode: 1/200, Reward: -0.19150040623533374, Epsilon: 1.0
Episode: 2/200, Reward: -0.12025790476767195, Epsilon: 1.0
Episode: 3/200, Reward: -0.08389877927543765, Epsilon: 1.0
Episode: 4/200, Reward: -0.20611403522051927, Epsilon: 1.0
Episode: 5/200, Reward: -0.09322086586159739, Epsilon: 1.0
Episode: 6/200, Reward: -0.25861045648067305, Epsilon: 1.0
Episode: 7/200, Reward: -0.1670877417857197, Epsilon: 1.0
Episode: 8/200, Reward: -0.08850113459930728, Epsilon: 1.0
Episode: 9/200, Reward: -0.3299641200841741, Epsilon: 1.0
Episode: 10/200, Reward: -0.16688735926847362, Epsilon: 1.0
Episode: 11/200, Reward: -0.0159704684574664, Epsilon: 1.0
Episode: 12/200, Reward: -0.2969677080757567, Epsilon: 1.0
Episode: 13/200, Reward: -0.16688735926847362, Epsilon: 1.0
Episode: 14/200, Reward: -0.15362957393230753, Epsilon: 1.0
Episode: 15/200, Reward: -0.07165833169325278, Epsilon: 1.0
Episode: 16/200, Reward: -0.1207016012232477, Epsilon: 1.0
Episode: 17/200, Reward: -0.05641786728972902, Epsilon

Episode: 51/200, Reward: -0.25861045648067305, Epsilon: 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

Episode: 101/200, Reward: -0.08480449398889454, Epsilon: 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s

Episode: 151/200, Reward: -0.2560617124459403, Epsilon: 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [6]:
state = env.reset()
state = np.reshape(state, [1, state_size])
done = False
while not done:
    action = agent.act(state)
    next_state, reward, done = env.step(action)
    state = np.reshape(next_state, [1, state_size])
    print(f"Action: {'Mobil' if action == 0 else 'Motor'}, Reward: {reward}")


Action: Motor, Reward: -0.13663065218012962


In [7]:
import folium
import pandas as pd

# Membaca dataset
data = pd.read_csv('surabaya_routes_with_vehicle_times.csv')

# Definisikan titik awal dan tujuan
start_name = "Stasiun Gubeng"
end_name = "Tunjungan Plaza"

# Filter data berdasarkan titik awal dan tujuan
filtered_data = data[(data['Start_Name'] == start_name) & (data['End_Name'] == end_name)]

# Urutkan berdasarkan waktu perjalanan untuk mobil atau motor (pilih yang lebih cepat)
filtered_data_sorted_mobil = filtered_data.sort_values(by='Travel_Time_Mobil_min', ascending=True)
filtered_data_sorted_motor = filtered_data.sort_values(by='Travel_Time_Motor_min', ascending=True)

# Pilih 2 rute tercepat (mobil atau motor)
best_routes_mobil = filtered_data_sorted_mobil.head(2)
best_routes_motor = filtered_data_sorted_motor.head(2)

# Lokasi untuk peta
locations = {
    "Stasiun Gubeng": [-7.257472, 112.752088],
    "Tunjungan Plaza": [-7.266735, 112.736115]
}

# Membuat peta menggunakan Folium
m = folium.Map(location=locations["Stasiun Gubeng"], zoom_start=13)

# Menambahkan marker untuk titik awal dan titik tujuan
folium.Marker(locations["Stasiun Gubeng"], popup="Stasiun Gubeng", icon=folium.Icon(color="red")).add_to(m)
folium.Marker(locations["Tunjungan Plaza"], popup="Tunjungan Plaza", icon=folium.Icon(color="blue")).add_to(m)

# Visualisasikan rute berdasarkan hasil optimisasi DQN untuk Mobil
for idx, row in best_routes_mobil.iterrows():
    start = row['Start_Name']
    end = row['End_Name']
    route_info = f"Route Mobil: {start} -> {end} (Time: {row['Travel_Time_Mobil_min']} mins)"
    folium.PolyLine(
        locations=[locations[start], locations[end]],
        color="green",  # Warna untuk mobil
        weight=5,
        opacity=0.7,
        tooltip=route_info  # Menampilkan waktu dan jenis kendaraan
    ).add_to(m)

# Visualisasikan rute berdasarkan hasil optimisasi DQN untuk Motor
for idx, row in best_routes_motor.iterrows():
    start = row['Start_Name']
    end = row['End_Name']
    route_info = f"Route Motor: {start} -> {end} (Time: {row['Travel_Time_Motor_min']} mins)"
    folium.PolyLine(
        locations=[locations[start], locations[end]],
        color="orange",  # Warna untuk motor
        weight=5,
        opacity=0.7,
        tooltip=route_info  # Menampilkan waktu dan jenis kendaraan
    ).add_to(m)

# Menambahkan marker tambahan untuk rute yang lebih detail
for idx, row in best_routes_mobil.iterrows():
    start = row['Start_Name']
    end = row['End_Name']
    folium.Marker(
        location=locations[start],
        popup=f"Start: {start}\n{row['Travel_Time_Mobil_min']} mins to {end}",
        icon=folium.Icon(color="green", icon="info-sign")
    ).add_to(m)
    folium.Marker(
        location=locations[end],
        popup=f"End: {end}\n{row['Travel_Time_Mobil_min']} mins from {start}",
        icon=folium.Icon(color="green", icon="info-sign")
    ).add_to(m)

for idx, row in best_routes_motor.iterrows():
    start = row['Start_Name']
    end = row['End_Name']
    folium.Marker(
        location=locations[start],
        popup=f"Start: {start}\n{row['Travel_Time_Motor_min']} mins to {end}",
        icon=folium.Icon(color="orange", icon="info-sign")
    ).add_to(m)
    folium.Marker(
        location=locations[end],
        popup=f"End: {end}\n{row['Travel_Time_Motor_min']} mins from {start}",
        icon=folium.Icon(color="orange", icon="info-sign")
    ).add_to(m)

# Simpan peta sebagai file HTML
m.save("optimized_routes_detailed.html")
print("Routes visualized in 'optimized_routes_detailed.html'")


Routes visualized in 'optimized_routes_detailed.html'
